In [1]:
import numpy as np
import pandas as pd

from pyspark.sql import SQLContext

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import matplotlib.pylab as plt

import os
os.chdir("/home/jovyan/work")

import warnings
warnings.filterwarnings('ignore')
import DataCollector as collector
dc = collector.DataCollector()
import TrainingConfigurations as configurations
t = configurations.TrainingConfigurations()
import FeaturePreparer as preparer
fp = preparer.FeaturePreparer()

dc._init_()
data = dc.loadData()
spark = dc.getSparkSession()

In [2]:
#data = dc.negativeValues(data)

In [3]:
data.show(10)

+-------------+--------+-----------------+---------------+------------+----+-----+---+----+------+-------+
|   CSTARTTIME|CSERVICE|CSENDERENDPOINTID|CSENDERPROTOCOL|CINBOUNDSIZE|year|month|day|hour|minute|outcome|
+-------------+--------+-----------------+---------------+------------+----+-----+---+----+------+-------+
|1572335390398|       5|             4946|              0|        6545|2019|   10| 29|   8|    49|      1|
|1572336843110|       6|             2718|              0|         193|2019|   10| 29|   9|    14|      1|
|1572346925970|       6|             2718|              0|         562|2019|   10| 29|  12|     2|      1|
|1572363173767|       6|             3978|              0|         195|2019|   10| 29|  16|    32|      1|
|1572424572659|       6|             2276|              0|         195|2019|   10| 30|   9|    36|      1|
|1572448533657|       6|             3316|              4|       18102|2019|   10| 30|  16|    15|      1|
|1572504329226|       6|             

In [4]:
import shutil
import os

path = os.path.join('new_message_tracking/data/')
shutil.rmtree(path)
os.mkdir(path)

In [ ]:
from xgboost.spark import SparkXGBClassifier
from pyspark.ml.feature import VectorAssembler
import shutil
import os

counter = 1
n_splits = 10000
count = data.count()
each_len = int(count/n_splits)
modelPath = 'new_message_tracking/models/model_1.json'
os.mkdir(modelPath)

data = fp.prepare_data_for_training_1(data)

while counter < n_splits:
    df_py = data.limit(each_len)
    data = data.subtract(df_py)

    train_df, test_df = df_py.randomSplit([0.8, 0.2], seed=1)
    train_df.write.mode('append').parquet('new_message_tracking/data/train.parquet')
    test_df.write.mode('append').parquet('new_message_tracking/data/test.parquet')

    if counter == 1:
        classifier = SparkXGBClassifier(
          features_col="features",
          label_col="outcome",
          num_workers=2,
        )

    else:
        classifier = classifier.load(modelPath)
        
    counter = counter +1
    model = classifier.fit(train_df)
    shutil.rmtree(modelPath)
    classifier.save(modelPath)
    
#    if counter == 2:
#        break

In [ ]:
test_data=spark.read.parquet('new_message_tracking/data/train.parquet')
train_data=spark.read.parquet('new_message_tracking/data/train.parquet')

In [ ]:
predictionAndLabels = test_data.map(lambda lp: (float(model.transform(lp.select('features'))), lp.outcome))

# Instantiate metrics object
metrics = BinaryClassificationMetrics(predictionAndLabels)

# Area under precision-recall curve
print("Area under PR = %s" % metrics.areaUnderPR)

# Area under ROC curve
print("Area under ROC = %s" % metrics.areaUnderROC)

In [ ]:
import shap

shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(train_data)

In [ ]:
shap.force_plot(explainer.expected_value, shap_values[0,:], train_data.iloc[0,:])

In [ ]:
shap.force_plot(explainer.expected_value, shap_values, train_data)

In [ ]:
shap.summary_plot(shap_values, train_data)

In [ ]:
shap.summary_plot(shap_values, train_data, plot_type="bar")